In [1]:
# import xml.etree.ElementTree as ET
import requests
import os
import pandas as pd


In [2]:
oc = os.getenv('OC', '')

# 법령

In [211]:
df = pd.read_csv("data/법령검색목록_법령_건축.csv", skiprows=1)
print(len(df))
df.head()

42


,순번,법령MST,소관부처코드,소관부처명,법령ID,법령명,공포일자,공포번호,시행일자,법령구분코드,법령구분명,법령분야코드,법령분야명
0,1,276923,1613000,국토교통부,10594,건축기본법,20251001,제21065호,20251001,A0002,법률,34030000,第3章 建 築
1,2,277915,1613000,국토교통부,10787,건축기본법 시행령,20251001,제35811호,20251001,A0007,대통령령,34030000,第3章 建 築
2,3,266691,1613000,국토교통부,13478,건축물관리법,20241203,제20549호,20250604,A0002,법률,34030000,第3章 建 築
3,4,271337,1613000,국토교통부,13742,건축물관리법 시행령,20250527,제35549호,20250604,A0007,대통령령,34030000,第3章 建 築
4,5,271531,1613000,국토교통부,13747,건축물관리법 시행규칙,20250602,제1495호,20250602,A0103,국토교통부령,34030000,第3章 建 築


In [4]:
import json
from langchain_core.documents import Document

def load_documents_from_jsonl(file_path):
    """
    JSONL 파일에서 Document 객체 리스트를 불러옵니다.
    """
    documents = []
    
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
                
            data = json.loads(line)
            doc = Document(
                page_content=data["page_content"],
                metadata=data["metadata"],
                id=data["id"]
            )
            documents.append(doc)
    
    return documents

In [5]:
documents = load_documents_from_jsonl("documents.jsonl")
documents

[Document(id='2019610_clause_1', metadata={'자치법규ID': '2019610', '자치법규명': '가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례', '자치법규일련번호': '1298118', '지자체기관명': '경기도 가평군', '시행일자': '20170731', '제개정정보': '일부개정', '자치법규종류': 'C0001', '공포번호': '2623', '전화번호': '031-580-2430', '담당부서명': '건설도시국 건설과', '자치법규발의종류': '', '공포일자': '20170731', '약어': {}, 'links': [{'자연재해대책법': ['제27조제2항']}]}, page_content='제1조(목적) 이 조례는 「자연재해대책법」 제27조제2항의 규정에 따라 건축물관리자의 제설·제빙에 관한 사항을 구체적으로 정함으로써 눈 또는 얼음으로 인한 주민의 불편을 최소화하고, 안전을 꾀함을 목적으로 한다.'),
 Document(id='2019610_clause_2', metadata={'자치법규ID': '2019610', '자치법규명': '가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례', '자치법규일련번호': '1298118', '지자체기관명': '경기도 가평군', '시행일자': '20170731', '제개정정보': '일부개정', '자치법규종류': 'C0001', '공포번호': '2623', '전화번호': '031-580-2430', '담당부서명': '건설도시국 건설과', '자치법규발의종류': '', '공포일자': '20170731', '약어': {}, 'links': [{'도로법': [], '농어촌도로정비법': [], '자연재해대책법 시행령': ['제22조의8']}]}, page_content='제2조(정의) 이 조례에서 사용하는 용어의 뜻은 다음과 같다. <개정 2017.7.31.>1. “도로”라 함은 「도로법」에 따른 도로, 그 밖에 일반 교통에 사용되는 모든 곳을 말한다.2. “차도”라 함은 연

In [6]:
df.iloc[0,:]

순번               1
법령MST       276923
소관부처코드     1613000
소관부처명        국토교통부
법령ID         10594
법령명          건축기본법
공포일자      20251001
공포번호       제21065호
시행일자      20251001
법령구분코드       A0002
법령구분명           법률
법령분야코드    34030000
법령분야명      第3章 建 築
Name: 0, dtype: object

In [222]:
id = df["법령ID"][3]
url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=eflaw&ID={id}&type=JSON"
response = requests.get(url)
response.raise_for_status()

data = response.json()

In [223]:
id = "13742"
url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=eflaw&ID={id}&type=JSON"
response = requests.get(url)
response.raise_for_status()

data = response.json()
content = data["법령"]["조문"]["조문단위"]

In [224]:
data["법령"].keys()

dict_keys(['개정문', '법령키', '별표', '기본정보', '부칙', '조문', '제개정이유'])

In [225]:
data["법령"]["별표"]

{'별표단위': [{'별표제목문자열': '건축물관리점검기관의 요건(제12조제2항 관련)',
   '별표키': '000100E',
   '별표내용': [['■ 건축물관리법 시행령 [별표 1] <개정 2021. 1. 5.>',
     '                                                                                      ',
     '  건축물관리점검기관의 요건(제12조제2항 관련)                                         ',
     '                                                                                      ',
     '  1. 정기점검, 긴급점검 및 소규모 노후 건축물등 점검 기관                             ',
     '    가. 기술인력: 점검대상 규모에 따른 다음의 기술인력을 모두 갖출 것                 ',
     '┌────────────────┬────────────────────────┐',
     '│구분                            │점검대상 규모                                   │',
     '│                                ├──────┬──────┬──────────┤',
     '│                                │연면적      │연면적      │연면적              │',
     '│                                │3천제곱미터 │3천제곱미터 │1만제곱미터 이상    │',
     '│                                │미만        │이상        │                    │',
     '│                

In [8]:
# 현행법령(시행일): http://www.law.go.kr/DRF/lawService.do?OC=test&target=eflaw&ID=1747&type=JSON
# 자체법규: http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=ordin&ID={id}&type=JSON

In [8]:
data["법령"].keys()

dict_keys(['개정문', '법령키', '기본정보', '부칙', '조문', '제개정이유'])

In [9]:
data["법령"]["기본정보"]

{'법령명_한글': '건축기본법',
 '공포번호': '21065',
 '전화번호': '044-201-3766',
 '언어': '한글',
 '제개정구분': '타법개정',
 '법령ID': '010594',
 '공동부령정보': '',
 '소관부처': {'content': '국토교통부', '소관부처코드': '1613000'},
 '공포법령여부': 'N',
 '법종구분': {'content': '법률', '법종구분코드': 'A0002'},
 '시행일자': '20251001',
 '연락부서': {'부서단위': {'부서연락처': '044-201-3766',
   '부서키': '388805',
   '부서명': '건축정책과',
   '소관부처명': '국토교통부',
   '소관부처코드': '1613000'}},
 '법령명_한자': '건축기본법',
 '법령명약칭': '',
 '공포일자': '20251001',
 '편장절관': '34030000'}

In [50]:
content = data["법령"]["조문"]["조문단위"]
content[0]

{'조문번호': '1',
 '조문제개정유형': '타법개정',
 '조문시행일자': '20251001',
 '조문변경여부': 'N',
 '조문이동이전': '',
 '조문키': '0001000',
 '조문내용': '            제1장 총칙',
 '조문제목': '',
 '조문이동이후': '',
 '조문여부': '전문'}

In [25]:
for cc in content:
    if "조문참고자료" in cc.keys():
        print(cc)
        # print(cc["조문번호"], cc["조문참고자료"])

{'조문번호': '25', '조문제개정유형': '타법개정', '조문시행일자': '20251001', '조문변경여부': 'N', '조문이동이전': '', '조문참고자료': '[본조신설 2015.8.11]', '조문키': '0025001', '항': [{'항번호': '①', '항내용': '① 국토교통부장관은 건축물의 설계, 시공, 공사감리 및 유지ㆍ관리 등과 관련된 「건축법」 및 그 관계 법령, 행정규칙 및 조례 등의 규정(이하 이 조에서 "건축물 관련 규정"이라 한다)을 종합적으로 안내하고, 합리적으로 운용하기 위하여 건축물 관련 규정을 관장하는 중앙행정기관의 장 및 지방자치단체의 장과 협의하여 건축물 관련 규정을 통합한 한국건축규정(이하 "한국건축규정"이라 한다)을 공고할 수 있다. <개정 2021.7.27>'}, {'항번호': '②', '항내용': '② 국토교통부장관은 제1항에 따라 공고된 한국건축규정을 체계적으로 관리하고 국민에게 제공하기 위하여 한국건축규정 정보체계를 구축ㆍ운영하여야 한다. <개정 2021.7.27>'}, {'항번호': '③', '항내용': '③ 국토교통부장관은 제2항에 따른 한국건축규정 정보체계를 효율적으로 관리하고 대국민서비스를 제고하기 위하여 대통령령으로 정하는 기관 또는 단체에 한국건축규정 정보체계의 운영을 위탁할 수 있다. <신설 2021.7.27>'}, {'항번호': '④', '항내용': '④ 중앙행정기관의 장 및 지방자치단체의 장은 소관하는 건축물 관련 규정이 제정, 개정 또는 폐지되는 경우에는 대통령령으로 정하는 바에 따라 한국건축규정에 반영되도록 조치하여야 한다. <신설 2021.7.27>'}], '조문내용': '제25조(한국건축규정의 공고 등)', '조문제목': '한국건축규정의 공고 등', '조문이동이후': '', '조문여부': '조문'}
{'조문번호': '26', '조문제개정유형': '타법개정', '조문시행일자': '20251001', '조문변경여부': 'N', '조문이동이전': '', '조문참고자료': '[본조신설 2

In [68]:
content[0]

{'조문번호': '1',
 '조문제개정유형': '타법개정',
 '조문시행일자': '20251001',
 '조문변경여부': 'N',
 '조문이동이전': '',
 '조문키': '0001000',
 '조문내용': '            제1장 총칙',
 '조문제목': '',
 '조문이동이후': '',
 '조문여부': '전문'}

In [69]:
content[1]

{'조문번호': '1',
 '조문제개정유형': '타법개정',
 '조문시행일자': '20251001',
 '조문변경여부': 'N',
 '조문이동이전': '',
 '조문키': '0001001',
 '조문내용': '제1조(목적) 이 법은 건축에 관한 국가 및 지방자치단체와 국민의 책무를 정하고 건축정책의 수립ㆍ시행 등을 규정하여 건축문화를 진흥함으로써 국민의 건전한 삶의 영위와 복리향상에 이바지함을 목적으로 한다.',
 '조문제목': '목적',
 '조문이동이후': '',
 '조문여부': '조문'}

In [42]:
ctx["항"]

{'호': [{'호번호': '1.', '호내용': '1.  국민의 안전ㆍ건강 및 복지에 직접 관련된 생활공간의 조성'},
  {'호번호': '2.', '호내용': '2.  사회의 다양한 요구를 조정하고 수용하며 경제활동의 토대가 되는 공간환경의 조성'},
  {'호번호': '3.', '호내용': '3.  지역의 고유한 생활양식과 역사를 반영하고 미래세대에 계승될 문화공간의 창조 및 조성'}]}

In [34]:
ctx

{'조문번호': '2',
 '조문제개정유형': '타법개정',
 '조문시행일자': '20251001',
 '조문변경여부': 'N',
 '조문이동이전': '',
 '조문키': '0002001',
 '항': {'호': [{'호번호': '1.', '호내용': '1.  국민의 안전ㆍ건강 및 복지에 직접 관련된 생활공간의 조성'},
   {'호번호': '2.', '호내용': '2.  사회의 다양한 요구를 조정하고 수용하며 경제활동의 토대가 되는 공간환경의 조성'},
   {'호번호': '3.',
    '호내용': '3.  지역의 고유한 생활양식과 역사를 반영하고 미래세대에 계승될 문화공간의 창조 및 조성'}]},
 '조문내용': '제2조(기본이념) 이 법은 국가 및 지방자치단체와 국민의 공동의 노력으로 다음 각 호와 같은 건축의 공공적 가치를 구현함을 기본이념으로 한다.',
 '조문제목': '기본이념',
 '조문이동이후': '',
 '조문여부': '조문'}

In [40]:
ctx["항"]

{'호': [{'호번호': '1.', '호내용': '1.  국민의 안전ㆍ건강 및 복지에 직접 관련된 생활공간의 조성'},
  {'호번호': '2.', '호내용': '2.  사회의 다양한 요구를 조정하고 수용하며 경제활동의 토대가 되는 공간환경의 조성'},
  {'호번호': '3.', '호내용': '3.  지역의 고유한 생활양식과 역사를 반영하고 미래세대에 계승될 문화공간의 창조 및 조성'}]}

In [90]:
ctx_list = []
ctx_org = []
meta_key = []
header = ""
ctx_meta = {}

for ctx in content:
    ctx_txt = ctx["조문내용"].strip()
    if ctx["조문여부"] == "조문": # 조
        # txt = header + " " + ctx_txt
        txt = ctx_txt
        if "항" in ctx.keys():
            if type(ctx["항"]) == type([]):
                for hang in ctx["항"]:
                    txt += "\n" + hang["항내용"].strip()
            else:
                for ho in ctx["항"]["호"]:
                    txt += "\n" + ho["호내용"].strip()
        ctx_list.append(txt.strip())
        
        ctx_meta["조문번호"] = ctx["조문번호"]
        ctx_meta["조문제목"] = ctx["조문제목"]
        if "항" in ctx.keys():
            ctx_meta["항"] = True
        else:
            ctx_meta["항"] = False

        # header = ""
        meta_key.append(ctx_meta.copy())  # .copy()로 복사본 추가
    else: # 장, 항, 
        header = "[" + ctx_txt + "]"
        # ctx_meta["조문번호"] = ctx["조문번호"]
        ctx_meta["장번호"] = ctx_txt.split(" ")[0]
        ctx_meta["장제목"] = " ".join(ctx_txt.split(" ")[1:])


    ctx_org.append(ctx_txt)
    # meta_key.append(ctx_meta.copy())  # .copy()로 복사본 추가
ctx_list

['제1조(목적) 이 법은 건축에 관한 국가 및 지방자치단체와 국민의 책무를 정하고 건축정책의 수립ㆍ시행 등을 규정하여 건축문화를 진흥함으로써 국민의 건전한 삶의 영위와 복리향상에 이바지함을 목적으로 한다.',
 '제2조(기본이념) 이 법은 국가 및 지방자치단체와 국민의 공동의 노력으로 다음 각 호와 같은 건축의 공공적 가치를 구현함을 기본이념으로 한다.\n1.  국민의 안전ㆍ건강 및 복지에 직접 관련된 생활공간의 조성\n2.  사회의 다양한 요구를 조정하고 수용하며 경제활동의 토대가 되는 공간환경의 조성\n3.  지역의 고유한 생활양식과 역사를 반영하고 미래세대에 계승될 문화공간의 창조 및 조성',
 '제3조(정의) 이 법에서 사용하는 용어의 정의는 다음과 같다.\n1.  "건축물"이란 토지에 정착하는 공작물 중 지붕과 기둥 또는 벽이 있는 것과 이에 부수되는 시설물을 말한다.\n2.  "공간환경(空間環境)"이란 건축물이 이루는 공간구조ㆍ공공공간 및 경관을 말한다.\n3.  "공공공간(公共空間)"이란 가로ㆍ공원ㆍ광장 등의 공간과 그 안에 부속되어 공중(公衆)이 이용하는 시설물을 말한다.\n4.  "건축디자인"이란 품격과 품질이 우수한 건축물과 공간환경의 조성으로 건축의 공공성을 실현하기 위하여 건축물과 공간환경을 기획ㆍ설계하고 개선하는 행위를 말한다.\n5.  "품격"이란 주변환경과의 관계, 규모, 형태, 구조, 재료, 시공수준 등을 통하여 그 목적과 지역의 정체성을 창출할 수 있는 적절성을 말한다.\n6.  "품질"이란 안전, 보건, 기능, 쾌적, 자원절약과 재활용 등의 객관적 성능을 말한다.\n7.  "건축"이란 건축물과 공간환경을 기획, 설계, 시공 및 유지관리하는 것을 말한다.',
 '제4조(국가 및 지방자치단체의 책무)\n① 국가는 품격과 품질이 우수한 건축물과 공간환경을 조성하기 위한 종합적인 건축정책을 수립ㆍ시행하여야 하며, 지방자치단체는 국가의 건축정책에 맞추어 지역의 실정에 부합하는 건축정책을 수립ㆍ시행하여야 한다.\n② 국가, 지방자치단체 

In [91]:
meta_key

[{'장번호': '제1장', '장제목': '총칙', '조문번호': '1', '조문제목': '목적', '항': False},
 {'장번호': '제1장', '장제목': '총칙', '조문번호': '2', '조문제목': '기본이념', '항': True},
 {'장번호': '제1장', '장제목': '총칙', '조문번호': '3', '조문제목': '정의', '항': True},
 {'장번호': '제1장',
  '장제목': '총칙',
  '조문번호': '4',
  '조문제목': '국가 및 지방자치단체의 책무',
  '항': True},
 {'장번호': '제1장', '장제목': '총칙', '조문번호': '5', '조문제목': '국민의 의무', '항': True},
 {'장번호': '제1장', '장제목': '총칙', '조문번호': '6', '조문제목': '다른 법률과의 관계', '항': False},
 {'장번호': '제2장',
  '장제목': '건축정책의 기본방향',
  '조문번호': '7',
  '조문제목': '건축의 생활공간적 공공성 구현',
  '항': True},
 {'장번호': '제2장',
  '장제목': '건축정책의 기본방향',
  '조문번호': '8',
  '조문제목': '건축의 사회적 공공성 확보',
  '항': True},
 {'장번호': '제2장',
  '장제목': '건축정책의 기본방향',
  '조문번호': '9',
  '조문제목': '건축의 문화적 공공성 실현',
  '항': True},
 {'장번호': '제3장',
  '장제목': '건축정책의 수립',
  '조문번호': '10',
  '조문제목': '건축정책기본계획의 수립',
  '항': True},
 {'장번호': '제3장',
  '장제목': '건축정책의 수립',
  '조문번호': '11',
  '조문제목': '건축정책기본계획의 내용',
  '항': True},
 {'장번호': '제3장',
  '장제목': '건축정책의 수립',
  '조문번호': '12',
  '조문제목': '지역건축기본계획의 수립 등'

In [92]:
len(meta_key), len(ctx_list)

(26, 26)

In [100]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

model = ChatOllama(
    model="gpt-oss:120b-cloud",
    temperature=0.1,
    max_tokens = 1024,
    timeout=None,
    max_retries=2,
    reasoning = None,
)

system_message = """당신은 법률 문서 전문을 분석하여, 그 안에서 명시적으로 **참고해야 하는 다른 법률 문서와 해당 조항 번호**를 추출해야 합니다.

아래의 원칙을 반드시 지키세요:
1. 반드시 문서 내용에 명시적으로 등장한 법률명 또는 조항만 추출합니다.
2. 법률명이 명확히 언급되지 않은 경우, 절대 추측하지 않습니다.
3. 조항 번호나 별표만 언급된 경우, 해당 항목은 "Attachment" key에 넣습니다.
4. 출력은 반드시 **유효한 JSON 형식**으로 반환합니다.

다음은 약어와 실제 명칭입니다. 법률 문서 제목이나 조항명이 약어로 명시되어있는 경우, 실제 명칭으로 변환하여 반환하세요.
{abb}

출력 형식:
  "법률 문서 제목": ["관련 조항 번호", "..."],
  "Attachment": ["별표", "조항 번호", "..."]

주의:
- 각 리스트가 비어있다면 빈 배열([])로 반환합니다.
- 설명 문구, 해석, 자연어 문장은 절대 포함하지 마세요.
- "같은 법" 이나 "동법" 등의 표현은 실제 법률명으로 대체하세요.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", "{question}")
])

# 일반 Chain 생성
chain = prompt | model | JsonOutputParser()
chain

ChatPromptTemplate(input_variables=['abb', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['abb'], input_types={}, partial_variables={}, template='당신은 법률 문서 전문을 분석하여, 그 안에서 명시적으로 **참고해야 하는 다른 법률 문서와 해당 조항 번호**를 추출해야 합니다.\n\n아래의 원칙을 반드시 지키세요:\n1. 반드시 문서 내용에 명시적으로 등장한 법률명 또는 조항만 추출합니다.\n2. 법률명이 명확히 언급되지 않은 경우, 절대 추측하지 않습니다.\n3. 조항 번호나 별표만 언급된 경우, 해당 항목은 "Attachment" key에 넣습니다.\n4. 출력은 반드시 **유효한 JSON 형식**으로 반환합니다.\n\n다음은 약어와 실제 명칭입니다. 법률 문서 제목이나 조항명이 약어로 명시되어있는 경우, 실제 명칭으로 변환하여 반환하세요.\n{abb}\n\n출력 형식:\n  "법률 문서 제목": ["관련 조항 번호", "..."],\n  "Attachment": ["별표", "조항 번호", "..."]\n\n주의:\n- 각 리스트가 비어있다면 빈 배열([])로 반환합니다.\n- 설명 문구, 해석, 자연어 문장은 절대 포함하지 마세요.\n- "같은 법" 이나 "동법" 등의 표현은 실제 법률명으로 대체하세요.\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOllama(model='gpt-oss:120

In [101]:
model_abb = ChatOllama(
    model="gpt-oss:120b-cloud",
    temperature=0.1,
    max_tokens = 1024,
    timeout=None,
    max_retries=2,
    reasoning = None,
)

system_message_abb = """당신은 법률 문서 전문을 분석하여, 그 안에서 약어(또는 축약어)의 **원래 의미**를 추출해야 합니다.

#아래의 원칙을 반드시 지키세요:
1. 반드시 문서 내용에 **명시적으로 등장한 약어 또는 축약어만** 추출합니다.
2. 약어가 정의되지 않은 경우, 절대 추측하지 않습니다.
3. 약어가 정의된 문장은 보통 “(이하 ‘~’이라 한다)” 또는 “(이하 ‘~’라 한다)” 형태로 나타납니다.
4. 약어의 원래 의미에는 다음 요소들이 포함될 수 있습니다:
   - 관련 법령, 조항 번호, 시행령/시행규칙
   - “에 따라”, “에 의한”, “이 정하여 고시하는”, “으로 정하는”, “에서 규정한” 등의 조건문
   - 문장 속 수식어, 제약 조건 등
   이러한 조건은 **절대 생략하지 말고 그대로 포함하세요.**
5. 정의 구문에 "다만", "단서", "예외" 등이 이어지는 경우, 해당 조건도 실제 명칭에 반드시 포함합니다.
6. 출력은 반드시 **파싱 가능한 JSON 형식**으로 반환해야 합니다.

출력 형식:
  "약어": "원래 의미"

- 예시 1:
입력:
"제3조(정의) 이 조례에서 사용하는 용어의 뜻은 「건축물관리법」(이하 “법”이라 한다) 및 「건축법」에서 정하는 바에 따른다."

출력:
  "법": "건축물관리법"

- 예시 2:
입력:
"법 제42조제1항(단서 부분은 제외한다)에 따라 200제곱미터 이상인 대지에 건축을 하는 건축주는 다음 각 호의 어느 하나에 해당하는 조경면적(이하 “조경의무면적”이라 한다)에 법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준(이하 “조경기준”이라 한다)에 따라 조경의 조치를 하여야 한다. 다만, 다음 각 호의 기준보다 영 제27조제2항 각 호의 기준이 더 완화된 경우에는 그 기준에 따른다. <개정 2009.12.9., 2017.11.9.>1. 연면적(대지에 둘 이상의 건축물이 있는 경우에는 연면적의 합계로 한다. 이하 이 조에서 같다)이 2천제곱미터 이상인 건축물의 조경의무면적: 대지면적의 15퍼센트 이상2. 연면적이 1천제곱미터 이상 2천제곱미터 미만인 건축물의 조경의무면적 : 대지면적의 10퍼센트 이상3. 연면적이 1천제곱미터 미만인 건축물의 조경의무면적 : 대지면적의 5퍼센트 이상② 조경기준 제4조와 제5조제1항 및 제12조에서 정하는 조경면적 산정기준이나 조경면적의 배치기준에 따라 산정한 면적을 제1항에 따른 조경의무면적으로 산정한다. 다만, 다음 각 호에서 정하는 경우에는 그 기준에 따라 산정한 면적만을 조경의무면적으로 산정하되 조경의무면적의 2분의 1을 초과할 수 없다."

출력:
  "조경의무면적": "법 제42조제1항(단서 부분은 제외한다)에 따라 200제곱미터 이상인 대지에 건축을 하는 건축주는 다음 각 호의 어느 하나에 해당하는 조경면적에 법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준에 따라 조경의 조치를 하여야 하나, 다만 다음 각 호의 기준보다 영 제27조제2항 각 호의 기준이 더 완화된 경우에는 그 기준에 따른다.",
  "조경기준": "법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준"

#출력 규칙:
- 반드시 유효한 "JSON" 형식으로만 출력하세요.
- 약어가 전혀 없을 경우에는 정확히 빈 JSON 객체만 출력하세요.
 """

prompt_abb = ChatPromptTemplate.from_messages([
    ("system", system_message_abb),
    ("user", "{question}")
])

# 일반 Chain 생성
chain_abb = prompt_abb | model_abb | JsonOutputParser()
chain_abb

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 법률 문서 전문을 분석하여, 그 안에서 약어(또는 축약어)의 **원래 의미**를 추출해야 합니다.\n\n#아래의 원칙을 반드시 지키세요:\n1. 반드시 문서 내용에 **명시적으로 등장한 약어 또는 축약어만** 추출합니다.\n2. 약어가 정의되지 않은 경우, 절대 추측하지 않습니다.\n3. 약어가 정의된 문장은 보통 “(이하 ‘~’이라 한다)” 또는 “(이하 ‘~’라 한다)” 형태로 나타납니다.\n4. 약어의 원래 의미에는 다음 요소들이 포함될 수 있습니다:\n   - 관련 법령, 조항 번호, 시행령/시행규칙\n   - “에 따라”, “에 의한”, “이 정하여 고시하는”, “으로 정하는”, “에서 규정한” 등의 조건문\n   - 문장 속 수식어, 제약 조건 등\n   이러한 조건은 **절대 생략하지 말고 그대로 포함하세요.**\n5. 정의 구문에 "다만", "단서", "예외" 등이 이어지는 경우, 해당 조건도 실제 명칭에 반드시 포함합니다.\n6. 출력은 반드시 **파싱 가능한 JSON 형식**으로 반환해야 합니다.\n\n출력 형식:\n  "약어": "원래 의미"\n\n- 예시 1:\n입력:\n"제3조(정의) 이 조례에서 사용하는 용어의 뜻은 「건축물관리법」(이하 “법”이라 한다) 및 「건축법」에서 정하는 바에 따른다."\n\n출력:\n  "법": "건축물관리법"\n\n- 예시 2:\n입력:\n"법 제42조제1항(단서 부분은 제외한다)에 따라 200제곱미터 이상인 대지에 건축을 하는 건축주는 다음 각 호의 어느 하나에 해당하는 조경면적(이하 “조경의무면적”이라 한다)에 법 제4

In [277]:
chain_abb.invoke({"question": "[제4장  건축물의 대지 및 도로] 제17조(대지의 조경)① 법 제42조제1항(단서 부분은 제외한다)에 따라 200제곱미터 이상인 대지에 건축을 하는 건축주는 다음 각 호의 어느 하나에 해당하는 조경면적(이하 “조경의무면적”이라 한다)에 법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준(이하 “조경기준”이라 한다)에 따라 조경의 조치를 하여야 한다. 다만, 다음 각 호의 기준보다 영 제27조제2항 각 호의 기준이 더 완화된 경우에는 그 기준에 따른다. <개정 2009.12.9., 2017.11.9.>1. 연면적(대지에 둘 이상의 건축물이 있는 경우에는 연면적의 합계로 한다. 이하 이 조에서 같다)이 2천제곱미터 이상인 건축물의 조경의무면적: 대지면적의 15퍼센트 이상2. 연면적이 1천제곱미터 이상 2천제곱미터 미만인 건축물의 조경의무면적 : 대지면적의 10퍼센트 이상3. 연면적이 1천제곱미터 미만인 건축물의 조경의무면적 : 대지면적의 5퍼센트 이상② 조경기준 제4조와 제5조제1항 및 제12조에서 정하는 조경면적 산정기준이나 조경면적의 배치기준에 따라 산정한 면적을 제1항에 따른 조경의무면적으로 산정한다. 다만, 다음 각 호에서 정하는 경우에는 그 기준에 따라 산정한 면적만을 조경의무면적으로 산정하되 조경의무면적의 2분의 1을 초과할 수 없다. <개정 2009.12.9., 2013.12.6., 2015.10.1>1. 시행규칙 제26조제2항에 따라 대지조성을 위한 성토부분·절토부분 또는 되메우기를 하지 아니하는 굴착부분의 비탈면(이하 “비탈면등”이라 한다)에 환경의 보전을 위한 조치를 하여야 하는 부분 중 대지의 가장자리의 비탈면등의 평균경사도가 15도 이상인 부분은 대지면적에는 산정하나 조경의무면적의 산정에서는 제외하고, 평균경사도가 15도 미만인 비탈면등에 환경의 보전을 위한 조치를 하거나 조경의 조치를 하는 부분의 2분의 1에 해당하는 면적만을 조경의무면적에 산정한다.2. 조경기준 제3조제9호에 따른 인공지반조경부분(지상1층의 조경부분은 제외한다)은 그 조경면적의 3분의 2에 해당하는 면적만을 조경의무면적으로 산정한다.③ 삭제 <2009.12.9.>④ 영 제27조제1항제5호에 따라 대지에 염분이 함유되어 있는 경우와 건축물 용도의 특성 상 조경의 조치가 곤란하거나 불합리한 경우로서 시장이 인정하면 다음 각 호의 어느 하나에 해당하는 건축물은 조경의 조치를 하지 않을 수 있다. 다만, 제4호부터 제6호까지의 규정에 해당하는 건축물의 대지 또는 옥상광장(옥상광장을 설치하는 경우만 해당한다)에 제1항 각 호의 어느 하나에 해당하는 조경의무면적의 2분의 1 이상에 해당하는 면적에 조경기준에 따라 식재를 하거나 조경시설을 설치하여야 한다. <개정 2009.12.9.>1.「주차장법」제2조에 따른 노외주차장과 주차전용건축물2. 운동경기장(건축물이 아닌 시설물)3. 공장과 창고(지구단위계획에 따라 녹지용지가 조성되었거나 조성계획이 있는 지구단위계획구역에 건축하려는 경우만 해당한다)4. 도매시장과 소매시장5. 염분이 함유되어 있는 대지에 건축하는 건축물6. 중심상업지역과 일반상업지역에서 300제곱미터 이하의 대지에 건축하는 건축물⑤ 영 제27조제1항제10호에 따라 조경의 조치를 하지 아니할 수 있는 건축물 중 건축조례로 정하는 건축물은 같은 호 각 목의 어느 하나에 해당하는 건축물만을 말한다. <개정 2009.12.9>⑥ 조경기준에도 불구하고 건축허가의 부관(附款)에 따라 조경의 조치를 하게 할 수 있다. <개정 2009.12.9.>"})

{'조경의무면적': '법 제42조제1항(단서 부분은 제외한다)에 따라 200제곱미터 이상인 대지에 건축을 하는 건축주는 다음 각 호의 어느 하나에 해당하는 조경면적(이하 "조경의무면적"이라 한다)에 법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준에 따라 조경의 조치를 하여야 한다. 다만, 다음 각 호의 기준보다 영 제27조제2항 각 호의 기준이 더 완화된 경우에는 그 기준에 따른다.',
 '조경기준': '법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준',
 '비탈면등': '시행규칙 제26조제2항에 따라 대지조성을 위한 성토부분·절토부분 또는 되메우기를 하지 아니하는 굴착부분의 비탈면(이하 "비탈면등"이라 한다)'}

## Chunk

In [212]:
from langchain_core.documents import Document
import json

def load_documents_from_jsonl(file_path):
    """
    JSONL 파일에서 Document 객체 리스트를 불러옵니다.
    
    Args:
        file_path: JSONL 파일 경로
        
    Returns:
        List[Document]: Document 객체 리스트
    """
    documents = []
    
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            # 빈 줄 건너뛰기
            if not line.strip():
                continue
                
            # JSON 파싱
            data = json.loads(line)
            
            # Document 객체 생성
            doc = Document(
                page_content=data["page_content"],
                metadata=data["metadata"],
                id=data["id"]
            )
            documents.append(doc)
    
    print(f"✅ 총 {len(documents)}개 Chunks 불러오기 완료")
    return documents

def save_documents(documents, file_path):
    """
    Document 리스트를 JSONL 파일로 저장합니다.
    """
    with open(file_path, "w", encoding="utf-8") as f:
        for doc in documents:
            json.dump({
                "id": doc.id,
                "page_content": doc.page_content,
                "metadata": doc.metadata
            }, f, ensure_ascii=False)
            f.write("\n")
    print(f"✅ {len(documents)}개 Chunks {file_path}에 저장 완료")


In [213]:
from tqdm import tqdm
from langchain_core.documents import Document
from collections import defaultdict
import requests
import json
import os
from pathlib import Path

def extract_clause_documents(df, oc, chain_abb, chain, 
                             doc_topic="law",
                             saved_file_name="documents.jsonl", 
                             checkpoint_file_name="processed_ids.txt",
                             checkpoint_dir="checkpoints"):
    """
    조항별로 메타데이터를 추출하여 Document 리스트를 생성합니다.
    중간 저장 기능 포함.
    """
    saved_file_name = f"documents/{doc_topic}/{saved_file_name}"
    checkpoint_dir = f"{checkpoint_dir}/{doc_topic}"

    # 체크포인트 디렉토리 생성
    Path(checkpoint_dir).mkdir(exist_ok=True, parents=True)
    Path(f"documents/{doc_topic}").mkdir(exist_ok=True, parents=True)

    # 이미 처리된 법규 ID 확인
    processed_ids = set()
    checkpoint_file = os.path.join(checkpoint_dir, checkpoint_file_name)
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, "r", encoding="utf-8") as f:
            processed_ids = set(line.strip() for line in f if line.strip())
        print(f"✅ 이미 처리된 법규: {len(processed_ids)}개")
    
    documents = []
    
    # 기존 documents.jsonl이 있으면 로드
    if os.path.exists(saved_file_name):
        print(f"기존 {saved_file_name} 로드 중...")
        documents = load_documents_from_jsonl(saved_file_name)
        # print(f"✅ 기존 문서의 조항 리스트 {len(documents)}개 로드 완료")
    
    # 처리할 법규만 필터링
    df_law_ids = [str(id) for id in df["법령ID"]]
    laws_to_process = [id for id in df_law_ids if id not in processed_ids]
    
    if not laws_to_process:
        print("✅ 모든 법규가 이미 처리되었습니다.")
        return documents

    print(f"🔄 처리할 법규: {len(laws_to_process)}개 (전체 {len(df_law_ids)}개 중)")

    for id in tqdm(laws_to_process, desc="법규 처리"):
        try:
            url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=eflaw&ID={id}&type=JSON"
            response = requests.get(url)
            response.raise_for_status()

            data = response.json()
            meta = data["법령"]["기본정보"]
            content = data["법령"]["조문"]["조문단위"]

            # --- 조문별 텍스트 추출 ---
            ctx_list = []
            meta_key = []
            ctx_meta = {}
            
            ctx_meta["장번호"] = "제0장"
            ctx_meta["장제목"] = "없음"

            for ctx in content:
                ctx_txt = ctx["조문내용"].strip()
                if ctx["조문여부"] == "조문":  # 조
                    txt = ctx_txt
                    if "항" in ctx.keys():
                        if type(ctx["항"]) == type([]):
                            for hang in ctx["항"]:
                                txt += "\n" + hang["항내용"].strip()
                        else:
                            for ho in ctx["항"]["호"]:
                                txt += "\n" + ho["호내용"].strip()
                    ctx_list.append(txt.strip())
                    
                    ctx_meta["조문번호"] = ctx["조문번호"]
                    ctx_meta["조문제목"] = ctx["조문제목"]
                    ctx_meta["항"] = "항" in ctx.keys()

                    meta_key.append(ctx_meta.copy())
                else:  # 장, 편 등
                    # ctx_meta = {}  # 초기화
                    ctx_meta["장번호"] = ctx_txt.split(" ")[0]
                    ctx_meta["장제목"] = " ".join(ctx_txt.split(" ")[1:])

            # --- 약어 추출 ---
            abb = {}
            clause_abb_list = []  # 각 조항의 약어를 저장
            for i, item in enumerate(ctx_list):
                if "이하" in item:
                    print(f"  🔤 약어 추출 중... ({i+1}/{len(ctx_list)})", end='\r')
                    q = ctx_list[i]
                    abb_output = chain_abb.invoke({"question": q})
                    
                    filtered_abb = {
                        k: v for k, v in abb_output.items()
                        if ((k in ["약어", ""] and v) or (k not in ["약어", ""])) and k != v
                    }
                    abb.update(filtered_abb)
                else:
                    filtered_abb = {}
                
                clause_abb_list.append(filtered_abb)

            # --- 조항별 Document 생성 ---
            cnt = 1
            law_documents = []  # 현재 법규의 문서들
            
            for i, item in enumerate(ctx_list):
                print(f"  🔗 links 추출 중... ({i+1}/{len(ctx_list)})", end='\r')
                chunk = item

                q = "법률 문서의 내용은 다음과 같습니다: " + ctx_list[i]
                links_llm = chain.invoke({"abb": abb, "question": q})

                # --- 메타데이터 생성 ---
                doc_meta = meta.copy()
                doc_meta["약어"] = clause_abb_list[i]
                doc_meta["content"] = meta_key[i]
                doc_meta["links"] = []
                
                # links 필터링 및 처리
                if links_llm:
                    clu_num = doc_meta["content"]["조문번호"]
                    clu_num = f"제{clu_num}조"
                    
                    # Attachment에서 현재 조문번호 제거
                    if 'Attachment' in links_llm and isinstance(links_llm['Attachment'], list):
                        # print(f"  기존 Attachment: {links_llm['Attachment']}")
                        links_llm['Attachment'] = [
                            item for item in links_llm['Attachment'] 
                            if item != clu_num
                        ]
                    
                    # 필터링: "Attachment"와 "법률 문서 제목"은 값이 있을 때만 포함
                    filtered_links = {
                        k: v for k, v in links_llm.items()
                        if (k in ["Attachment", "법률 문서 제목"] and v) 
                        or (k not in ["Attachment", "법률 문서 제목"])
                    }
                    
                    if filtered_links:
                        # print(f"  필터링된 links: {filtered_links}")
                        doc_meta["links"] = [filtered_links]

                # --- Document 객체 생성 ---
                doc = Document(
                    page_content=chunk,
                    metadata=doc_meta,
                    id=f"{meta['법령ID']}_clause_{cnt}"
                )
                cnt += 1
                law_documents.append(doc)
            
            # 현재 법규 문서들을 전체 리스트에 추가
            documents.extend(law_documents)
            
            # 처리 완료된 ID 기록
            processed_ids.add(id)
            # 매 법규마다 저장
            save_documents(documents, saved_file_name)

            with open(checkpoint_file, "a", encoding="utf-8") as f:
                f.write(f"{id}\n")
            
        
        except Exception as e:
            print(f"\n❌ 오류 발생 (법규 ID: {id}): {str(e)}")
            import traceback
            print(traceback.format_exc())
            # 오류 로그 저장
            with open(os.path.join(checkpoint_dir, "error_log.txt"), "a", encoding="utf-8") as f:
                f.write(f"{id}: {str(e)}\n{traceback.format_exc()}\n")
            continue

    # 최종 저장
    save_documents(documents, saved_file_name)
    print(f"\n✅ 최종 저장: {len(documents)}개 문서 완료")
    
    return documents


In [214]:
def create_merged_document(law_id, chapter_idx, chunk_id, content, base_meta, links_dict, abb, mg_content):
    """
    병합된 Document 객체를 생성합니다.
    """
    metadata = base_meta.copy()
    
    # links 정리 (set을 list로 변환)
    if links_dict:
        metadata["links"] = [{k: sorted(list(v)) for k, v in links_dict.items()}]
    
    # 약어 추가
    if abb:
        metadata["약어"] = abb
    metadata["content"] = mg_content
    doc = Document(
        page_content=content,
        metadata=metadata,
        id=f"{law_id}_chapter{chapter_idx}_chunk{chunk_id}"
    )
    
    return doc

In [215]:
def merge_documents_by_chapter(documents, min_len=300, max_len=600, 
                               doc_topic="law",
                               merged_file_name="documents_merged.jsonl",
                               checkpoint_file_name="merged_ids_law.txt",
                               checkpoint_dir="checkpoints", batch_size=1):
    """
    조항별 Document를 장 단위로 병합하여 적절한 크기의 청크를 생성합니다.
    중간 저장 기능 포함.
    """
    checkpoint_dir = f"{checkpoint_dir}/{doc_topic}"
    # 체크포인트 디렉토리 생성
    Path(checkpoint_dir).mkdir(exist_ok=True, parents=True)
    merged_file_name = f"documents/{doc_topic}/{merged_file_name}"
    
    # 이미 병합된 문서가 있으면 로드
    merged_docs = []
    if os.path.exists(merged_file_name):
        print(f"기존 {merged_file_name} 로드 중...")
        merged_docs = load_documents_from_jsonl(merged_file_name)
        print(f"✅ 기존 병합 문서 청크 리스트 {len(merged_docs)}개 로드 완료")
    
    # 이미 처리된 법규 ID 확인
    processed_law_ids = set()
    merge_checkpoint_file = os.path.join(checkpoint_dir, checkpoint_file_name)
    if os.path.exists(merge_checkpoint_file):
        with open(merge_checkpoint_file, "r", encoding="utf-8") as f:
            processed_law_ids = set(line.strip() for line in f if line.strip())
        print(f"✅ 이미 병합된 법규: {len(processed_law_ids)}개")
    
    # 1️⃣ Document들을 법령ID 기준으로 그룹화
    docs_by_law = defaultdict(list)
    for doc in documents:
        law_id = str(doc.metadata.get("법령ID"))
        if law_id not in processed_law_ids:
            docs_by_law[law_id].append(doc)
    
    if not docs_by_law:
        print("✅ 모든 법규가 이미 병합되었습니다.")
        return merged_docs

    # 2️⃣ 각 법령 문서별로 처리
    total_laws = len(docs_by_law)
    processed_count = 0
    
    print(f"🔄 병합할 법규: {total_laws}개")
    
    for law_id, doc_list in tqdm(docs_by_law.items(), desc="법령별 병합"):
        try:
            # ID를 숫자 기준으로 정렬 (2019610_clause_1, 2019610_clause_2, ...)
            def extract_clause_number(doc):
                # ID 형식: "2019610_clause_1"
                parts = doc.id.split("_")
                if len(parts) >= 3:
                    try:
                        return int(parts[2])  # clause 번호
                    except:
                        return 0
                return 0
            
            doc_list = sorted(doc_list, key=extract_clause_number)
            base_meta = {k: v for k, v in doc_list[0].metadata.items() 
                         if k not in ["links", "약어", "content"]}

            # 3️⃣ 장별로 그룹화
            chapters = []
            current_chapter = []
            current_header = ""
            
            for doc in doc_list:
                content = doc.page_content
                header = doc.metadata["content"]["장번호"]
                
                # 헤더가 바뀌면 새 장 시작
                if header and header != current_header:
                    if current_chapter:
                        chapters.append({
                            "header": current_header,
                            "docs": current_chapter
                        })
                    current_header = header
                    current_chapter = [doc]
                else:
                    current_chapter.append(doc)
            
            # 마지막 장 추가
            if current_chapter:
                chapters.append({
                    "header": current_header,
                    "docs": current_chapter
                })

            # 4️⃣ 각 장별로 청크 생성
            for chapter_idx, chapter in enumerate(chapters, start=1):
                법령명_한글 = base_meta.get("법령명_한글", "")
                
                # 장 헤더가 있으면 포함
                if chapter["header"] and chapter["header"] != "제0장":
                    base_header = f"[{법령명_한글}]\n{chapter['header']}"
                else:
                    base_header = f"[{법령명_한글}]"
                
                buffer = base_header
                merged_links_dict = defaultdict(set)
                merged_abb = {}
                merged_meta_contents = []
                chunk_id = 1
                
                for doc in chapter["docs"]:
                    content = doc.page_content
                    
                    # 장 헤더 제거 (조문 내용만 추출)
                    text = content
                    # if content.startswith("[") and "]" in content:
                    #     # [제1장 총칙] 제1조... 형태에서 제1조... 부분만 추출
                    #     end_idx = content.find("]")
                    #     potential_header = content[:end_idx+1]
                    #     # 장 헤더인 경우에만 제거
                    #     if "장" in potential_header or "편" in potential_header:
                    #         text = content[end_idx+1:].strip()
                    
                    # 현재 문서의 메타데이터 추출
                    links_list = doc.metadata.get("links", [])
                    abb = doc.metadata.get("약어", {})
                    meta_c = doc.metadata.get("content", {})
                    
                    # 버퍼에 추가할지 결정
                    potential_length = len(buffer) + len(text) + 1
                    
                    if potential_length <= max_len:
                        # 버퍼에 추가
                        buffer += f"\n{text}"
                        
                        # 링크 병합
                        for links in links_list:
                            for key, values in links.items():
                                if isinstance(values, list):
                                    merged_links_dict[key].update(values)
                                else:
                                    merged_links_dict[key].add(values)
                        
                        # 약어 및 메타 콘텐츠 병합
                        merged_abb.update(abb)
                        merged_meta_contents.append(meta_c)
                    else:
                        # 현재 버퍼가 min_len 미만이면 강제로 추가
                        if len(buffer) < min_len:
                            buffer += f"\n{text}"
                            
                            # 링크 병합
                            for links in links_list:
                                for key, values in links.items():
                                    if isinstance(values, list):
                                        merged_links_dict[key].update(values)
                                    else:
                                        merged_links_dict[key].add(values)
                            
                            # 약어 및 메타 콘텐츠 병합
                            merged_abb.update(abb)
                            merged_meta_contents.append(meta_c)
                        else:
                            # 현재 버퍼로 Document 생성
                            merged_doc = create_merged_document(
                                law_id=law_id,
                                chapter_idx=chapter_idx,
                                chunk_id=chunk_id,
                                content=buffer.strip(),
                                base_meta=base_meta,
                                links_dict=merged_links_dict,
                                abb=merged_abb,
                                mg_content=merged_meta_contents
                            )
                            merged_docs.append(merged_doc)
                            
                            # 새 버퍼 시작 (현재 문서로 시작)
                            chunk_id += 1
                            buffer = base_header + f"\n{text}"
                            merged_links_dict = defaultdict(set)
                            merged_abb = {}
                            merged_meta_contents = []
                            
                            # 현재 조항의 링크, 약어, 메타 콘텐츠 추가
                            for links in links_list:
                                for key, values in links.items():
                                    if isinstance(values, list):
                                        merged_links_dict[key].update(values)
                                    else:
                                        merged_links_dict[key].add(values)
                            merged_abb.update(abb)
                            merged_meta_contents.append(meta_c)
                
                # 마지막 버퍼 처리 (헤더만 있는 경우 제외)
                if buffer.strip() != base_header.strip():
                    merged_doc = create_merged_document(
                        law_id=law_id,
                        chapter_idx=chapter_idx,
                        chunk_id=chunk_id,
                        content=buffer.strip(),
                        base_meta=base_meta,
                        links_dict=merged_links_dict,
                        abb=merged_abb,
                        mg_content=merged_meta_contents
                    )
                    merged_docs.append(merged_doc)
            
            # 처리 완료된 법규 ID 기록
            processed_law_ids.add(str(law_id))
            with open(merge_checkpoint_file, "a", encoding="utf-8") as f:
                f.write(f"{str(law_id)}\n")
            
            processed_count += 1
            
            # batch_size개 법규마다 중간 저장
            if processed_count % batch_size == 0:
                save_documents(merged_docs, merged_file_name)
                print(f"\n💾 중간 저장: {len(merged_docs)}개 병합 문서, {processed_count}/{total_laws}개 법규 처리 완료")
        
        except Exception as e:
            print(f"\n❌ 병합 오류 (법규 ID: {law_id}): {str(e)}")
            import traceback
            print(traceback.format_exc())
            # 오류 로그 저장
            with open(os.path.join(checkpoint_dir, "merge_error_log.txt"), "a", encoding="utf-8") as f:
                f.write(f"{law_id}: {str(e)}\n{traceback.format_exc()}\n")
            continue

    # 최종 저장
    save_documents(merged_docs, merged_file_name)
    print(f"\n✅ 최종 병합 저장: {len(merged_docs)}개 문서 완료")
    
    return merged_docs


In [216]:
def save_documents(documents, file_path):
    """
    Document 리스트를 JSONL 파일로 저장합니다.
    """
    with open(file_path, "w", encoding="utf-8") as f:
        for doc in documents:
            json.dump({
                "id": doc.id,
                "page_content": doc.page_content,
                "metadata": doc.metadata
            }, f, ensure_ascii=False)
            f.write("\n")

In [217]:
def load_documents_from_jsonl(file_path):
    """
    JSONL 파일에서 Document 객체 리스트를 불러옵니다.
    """
    documents = []
    
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
                
            data = json.loads(line)
            doc = Document(
                page_content=data["page_content"],
                metadata=data["metadata"],
                id=data["id"]
            )
            documents.append(doc)
    
    return documents

In [218]:
# --- 메인 실행 ---
if __name__ == "__main__":
    # 1단계: 조항별 메타데이터 추출
    print("=" * 50)
    print("1단계: 조항별 메타데이터 추출 중...")
    print("=" * 50)
    clause_documents = extract_clause_documents(df[:100], oc, chain_abb, chain, "law", 
                                                "documents_law.jsonl", 
                                                "processed_ids_law.txt")
    print(f"\n✅ 조항별 문서 {len(clause_documents)}개 완료")

    # 2단계: 장별 병합
    print("\n" + "=" * 50)
    print("2단계: 장별 병합 중...")
    print("=" * 50)
    merged_documents = merge_documents_by_chapter(
        clause_documents, 
        min_len=300, 
        max_len=600,
        doc_topic = "law",
        merged_file_name = "documents_merged.jsonl",
        checkpoint_file_name = "merged_ids_law.txt",
        checkpoint_dir = "checkpoints",
        batch_size=100  # 100개 법규마다 저장
    )
    print(f"\n✅ 병합 문서 {len(merged_documents)}개 완료")

    # print("\n" + "=" * 50)
    # print("🎉 모든 작업 완료!")
    # print(f"조항별 문서: {len(clause_documents)}개 → documents.jsonl")
    # print(f"병합 문서: {len(merged_documents)}개 → documents_merged.jsonl")
    # print("=" * 50)

1단계: 조항별 메타데이터 추출 중...
✅ 이미 처리된 법규: 2개
기존 documents/law/documents_law.jsonl 로드 중...
🔄 처리할 법규: 40개 (전체 42개 중)


법규 처리:   0%|          | 0/40 [00:00<?, ?it/s]

법규 처리:   2%|▎         | 1/40 [05:58<3:53:03, 358.56s/it]

법규 처리:   2%|▎         | 1/40 [06:22<4:08:50, 382.83s/it]


KeyboardInterrupt: 